Test whether the Beckmann microfacet distribution is a true PDF (integrates to 1).

First, integrate over $\theta$ from 0 to 90 degrees. Multiply the result by $2 \pi$ to include the integral from 0 to $2 \pi$ over $\phi$.

In [1]:
import math
import vec3
import microfacet

alpha = .9
beckmann = microfacet.Beckmann(alpha, alpha)

pdf_sum = 0
di = math.radians(1)
for i in range(0,90):
    cosi = math.cos(math.radians(i))
    sini = math.sin(math.radians(i))
    pdf_sum += beckmann.Pdf(vec3.Vec3(sini, 0.0, cosi)) * sini * di # sini because we are integrating dw
print(2 * math.pi * pdf_sum)

0.9999373217072938


Repeat the integral by integrating over $d \cos \theta$. 

In [4]:
N = 100
dcosi = 1/N
pdf_sum = 0.0
for i in range(N):
    cosi = i/N
    sini = math.sqrt(1.-cosi*cosi)
    d = beckmann.Pdf(vec3.Vec3(sini, 0.0, cosi)) * dcosi
    pdf_sum += d
print (2 * math.pi * pdf_sum)

0.9876433968100947


In [ ]:
The brdf is expected to integrate to <= 1 over omega_o (not omega_h). I need to make more adjustment to the cell below to get this to be correct.
BRDF integration to close to 1 test:
I have fixed the brdf integration test to encompass the correct theta range and phi range for our setup, and the beckmann brdf integrates close to 1 for small alpha but starts to become less with increasing alpha.
At alpha = 9, it is .618 for example.

In [1]:
import brdf
import math
import vec3

alpha = .5
theBrdf = brdf.Brdf(alpha, alpha);


theta_o = math.pi /6;
phi_o = 0
wo = vec3.Vec3(0, math.sin(theta_o) * math.cos(phi_o), math.cos(theta_o))

phi_i = math.pi

pdf_total = 0
brdf_total = 0

for j in range (0,  360):
    phi_j = math.radians(j)
    dj = math.radians(j+1) - phi_j
    brdf_sum = 0
    pdf_sum = 0
    cos_phi = math.cos(phi_j)
    sin_phi = math.sin(phi_j)
    for i in range(0, 90):
        cos_theta_i = math.cos(math.radians(i))
        sin_theta_i = math.sin(math.radians(i))
        di = math.radians(i+1) - math.radians(i)
        mu = cos_theta_i 
        wi = vec3.Vec3(sin_theta_i * sin_phi, sin_theta_i * cos_phi, mu)
        (value, pdf) = theBrdf.Eval(wo, wi, False) 
        brdf_sum += value * sin_theta_i *  di
        pdf_sum += pdf * sin_theta_i * di
    brdf_total += brdf_sum * dj
    pdf_total += pdf_sum * dj
print(brdf_total, pdf_total)

0.9030309454861812 0.9184999308485973
